In [50]:
#dependencies
import os 
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import pymongo 
import time
import pandas as pd

In [51]:
#chromedriver setup
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [52]:
#URL of NASA Mars News Site
mars_news_url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(mars_news_url)
time.sleep(1)

In [53]:
#find title and paragraph of most recent article
html = browser.html
soup = bs(html, 'html.parser')
news_title = soup.find('div', class_='content_title').text
print(f' Headline: {news_title}')
news_p = soup.find('div', class_='article_teaser_body').text
print(f' Teaser: {news_p}')

 Headline: NASA InSight Lander 'Hears' Martian Winds 
 Teaser: Vibrations picked up by two spacecraft instruments have provided the first sounds of Martian wind.


In [54]:
# Bring in JPL URL
featured_image_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(featured_image_url)
time.sleep(1)

In [55]:
#Find full URL of featured image
html_image = browser.html
soup = bs(html_image, 'html.parser')
image = soup.find("img", class_="thumb")["src"]
featured_image_url = "https://www.jpl.nasa.gov" + image
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA22873-640x350.jpg


In [28]:
#Bring in twitter URL
weather_url = "https://twitter.com/marswxreport?lang=en"
browser.visit(weather_url)

In [41]:
#Find latest weather tweet
weather_html = browser.html
weather_soup = bs(weather_html, 'html.parser')
latest_tweets = soup.find_all('div', class_='js-tweet-text-container')
for tweet in latest_tweets: 
    mars_weather = tweet.find('p').text
    if 'Sol' and 'pressure' in mars_weather:
        print(f'Mars Weather: {mars_weather}')
        break
    else: 
        pass
        

Mars Weather: Sol 2242 (2018-11-26), high -2C/28F, low -70C/-93F, pressure at 8.48 hPa, daylight 06:29-18:45


In [47]:
#Bring in Mars facts URL
facts_url = 'http://space-facts.com/mars/'

In [49]:
#Read with pandas
mars_facts = pd.read_html(facts_url)
facts_df = mars_facts[0]
facts_df.columns = ['Parameter','Value']
facts_html = facts_df.to_html(header=False, index=False)
facts_df

,Parameter,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [65]:
#bring in hemisphere images URL
mars_hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
hemisphere_image_urls = []

In [75]:
#scrape hemisphere image URL's and append to dictionary
for i in range(1,9,2):
    hemisphere_image = {}
    
    browser.visit(mars_hemisphere_url)
    time.sleep(1)
    hemispheres_html = browser.html
    soup = bs(hemispheres_html, 'html.parser')
    hemisphere_links = hemispheres_soup.find_all('a', class_='product-item')
    hemisphere_title = hemisphere_links[i].text.strip('Enhanced')
    
    links = browser.find_by_css('a.product-item')
    links[i].click()
    time.sleep(1)
    browser.find_link_by_text('Sample').first.click()
    time.sleep(1)
    browser.windows.current = browser.windows[-1]
    hemisphere_image_html = browser.html
    browser.windows.current = browser.windows[0]
    browser.windows[-1].close()
    
    hemisphere_soup = bs(hemisphere_image_html, 'html.parser')
    path = hemisphere_soup.find('img')['src']

    print(hemisphere_title)
    hemisphere_image['title'] = hemisphere_title.strip()
    
    print(path)
    hemisphere_image['img_url'] = path

    hemisphere_image_urls.append(hemisphere_image)

Cerberus Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
Schiaparelli Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
Syrtis Major Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
Valles Marineris Hemisphere 
https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg
